# 0 Vorbereitungen

In [1]:
import sys
sys.path.append(r'/home/tomas/Workspace/geo-adjust')

In [2]:
import numpy as np
import pandas as pd
from tabulate import tabulate
import IPython as ipy
from IPython.display import display

from dev.ipy_utils import show_matrix, show_dataframe, show_colored_matrix

# 1 Ausgleichungsdaten laden
Im ersten Schritt müssen Koordinaten (Neu- und Festpunkte), Instrumentenspezifikationen und Messdaten importiert werden.

## 1.1 Import der Koordinaten 

ASCII-Datei mit den zu verwendenden Koordinaten einlesen. Die Datei beinhaltet beinhaltet sowohl **Festpunkte** und Näherungskoordinaten der **Neupunkte**.


* **Festpunkte** - Koordinaten sind *unveränderlich*, d.h. im stochastischen Modell als varianzfrei. Üben Zwang auf das Netz
* **Neupunkte**  - Koordinaten sind in der Ausgleichung zu bestimmen

Die folgende Datei stellt ein Beispiel dar, enthält Punktnummer, Rechtswert, Hochwert, Höhe (in diesem Beispiel nicht verwendet) und die Punktart (`F` für Festpunkt und `N` für Neupunkt):

In [3]:
with open(r'../data/netz/punkte3.txt', 'r') as f:
    print(f.read())

223	3221.87	5339139.20	255.67	F
254	3041.31	5341095.72	304.91	F
1247	4363.29	5338161.10	350.85	F
263	1940.59	5341369.08	272.56	F
240	1775.96	5340230.37	248.00	F
035	89.25	5348679.90	---	F
182	1578.03	5336740.04	302.26	F
237	1543.78	5340069.22	---	F
10	2764.45	5339745.891	---	N
11	2784.911	5339758.008	---	N
21	2785.338	5339771.040	200.049	F
22	2803.123	5339782.199	---	N
31	2763.125	5339759.121	---	N
32	2739.317	5339745.618	---	N
33	2750.963	5339726.163	---	N



Mit `pandas` lässt sich die Datei schnell und einfach einlesen. Mit den Parametern `header=None, names=['pt', 'y', 'x', 'h', 'art']` spezifiziert man, dass keine Spaltennamen in der Datei enthalten sind und definiert sie. Mit dem Parameter `na_values` kann man Zeichenketten definieren, die automatisch auf *Not-A-Number* (`NaN`) geparsed werden.

In [4]:
def read_punkte(file):
    """
    Reads coordinates of form East, North, Height (Y,X,H) respectively.
    Column order: Pointnumber, East, North, Height, Pointtype (Fixed or New)
    
    :param file: path to tab-separated file
    
    """
    pt_df = pd.read_csv(file, sep='\t', na_values=['---'],
                        header=None, names=['pt', 'y', 'x', 'h', 'art'])
    pt_df.set_index('pt', inplace=True)
    return pt_df

In [5]:
punkte = read_punkte(r'../data/netz/punkte3.txt')
punkte

,y,x,h,art
pt,,,,
223,3221.870,5339139.200,255.670,F
254,3041.310,5341095.720,304.910,F
1247,4363.290,5338161.100,350.850,F
263,1940.590,5341369.080,272.560,F
240,1775.960,5340230.370,248.000,F
35,89.250,5348679.900,NaN,F
182,1578.030,5336740.040,302.260,F
237,1543.780,5340069.220,NaN,F
10,2764.450,5339745.891,NaN,N


Das Resultat ist ein *Dataframe*, das viele Tools zu Data Management und Data Analysis bereitstellt z.B. Zugriff auf den Punkt mit Punktnummer '1247':

In [6]:
punkte.loc[1247]

y          4363.29
x      5.33816e+06
h           350.85
art              F
Name: 1247, dtype: object

Oder alle Neupunkte:

In [7]:
punkte[punkte.art=='N']

,y,x,h,art
pt,,,,
10,2764.450,5339745.891,NaN,N
11,2784.911,5339758.008,NaN,N
22,2803.123,5339782.199,NaN,N
31,2763.125,5339759.121,NaN,N
32,2739.317,5339745.618,NaN,N
33,2750.963,5339726.163,NaN,N


## 1.2 Instrumentenspezifikation

Für das stochastische Modell der Ausgleichung werden die Instrumentenspezifikationen der verwendeten Totalstationen benötigt. Eine ASCII-Datei könnte folgendermaßen aussehen un beinhaltet Instrumentenname, Seriennummer, Winkelgenauigkeit [mgon], Streckengenauigkeit[mm] und distanzabhängige Streckengenauigkeit [ppm]:

In [8]:
with open(r'../data/netz/instruments.txt', 'r') as f:
    print(f.read())

TS16-Luigi	3012219	0.3	2	1.5
TS16-Gian	3011447	0.3	2	1.5
MS50	368078	0.3	2	1.5


In [9]:
def read_instruments(file):
    """
    Reads instrument specifications file.
    Column order: InstrumentName, SerialNumber, StdDevAngle, StdDevDistance, StdDevDistance (ppm)
    
    :param file: path to tab-separated file    
    """
    
    idf = pd.read_csv(file, sep='\t',
                        header=None, names=['name', 'seriennr', 's_ri', 's_d', 's_dd'])
    idf.set_index('seriennr', inplace=True, drop=False)
    return idf

instrumente = read_instruments(r'../data/netz/instruments.txt')
instrumente

,name,seriennr,s_ri,s_d,s_dd
seriennr,,,,,
3012219,TS16-Luigi,3012219,0.3,2,1.5
3011447,TS16-Gian,3011447,0.3,2,1.5
368078,MS50,368078,0.3,2,1.5


Möchte man nun auf die Winkelgenauigkeit eines bestimmten Instrumentes zugreifen kann man das mit

In [10]:
instrumente.at[3012219,'s_ri']

0.3

## 1.3 Import der Messdaten

Die bereits in Geosi reduzierten Messdaten werden pro Instrument eingelesen. Die Datei ist nach Standpunkten sortiert; mehrere Aufstellungen auf einem Punkt werden in dieser Version nicht unterstützt. Aus Geosi exportierte Messdaten haben folgendes Format:

In [11]:
with open(r'../data/netz/obs_bu.txt', 'r') as f:
    for i in range(10):
        print(f.readline().rstrip())
    print('...')

Stand: 10
  11                      0.0000      23.778      -0.019
  223                    92.9000         ---         ---
  254                   346.9069         ---         ---
  33                    172.2053      23.898       1.104
Stand: 11
  10                      0.0000      23.779       0.019
  1247                  284.4015         ---         ---
  21                    136.1269      13.040       1.106
  263                   103.2949         ---         ---
...


Eine Messdatenzeile beinhaltet Zielpunkt, Richtung, Horizontalstrecke und Höhendifferenz. Das Messdatenfile wird zeilenweise prozessiert.

In [12]:
def read_geosi(file, snr):
    """
    Einleseroutine der Messdaten aus Geosi.

    - Datei ist nach Standpunkten sortiert
    - Zeilenweise die Datei durchgehen
    - eigene Funktion read_geosi()

    - Einlesen der Datein passiert Instrumentenweise

    - Satzmittel ist bereits in Geosi gerechnet
        Berücksichtigung mehrerer Aufstellungen pro Standpunkt sollte berücksichtigt werden
        (in Feldübung implementieren) > Spalte aufst = nummer der Aufstellung

    :return: Messdaten Array (von, nach, aufst, ri, sh, dh)
    """

    def prealloc_matrix(shape, columns=None, fill_with_nan=False):
        if columns is None:
            m = np.empty(shape, dtype=np.float64)
        else:
            m = np.empty(shape, dtype=columns)
        if fill_with_nan:
            m.fill(np.NaN)
        return m

    # preallocate data matrix
    drl = 100000
    data = prealloc_matrix((drl,), columns=[('von', np.int16), ('nach', np.int16), ('aufst', np.int16),
                                            ('ri', np.float64), ('sh', np.float64), ('dh', np.float64), ('instr', np.int32)])

    # working vars
    standpunkt = None
    aufstellung = 1
    i = 0

    with open(file, 'r') as fh:
        for line in fh:
            if line.startswith('Stand'):
                # new Standpunkt
                standpunkt = line[7:].strip()
            else:
                try:
                    entries = [np.nan if e == '---' else e for e in line.split()]
                    data[i] = tuple([standpunkt, entries[0].strip(), aufstellung,
                                     np.float64(entries[1]), np.float64(entries[2]),
                                     np.float64(entries[3]), int(snr)])
                    i += 1
                except:
                    pass

    return pd.DataFrame(data[:i])

In [13]:
data = read_geosi(r'../data/netz/obs_bu.txt', 3012219)
data.loc[(data.von==10) | (data.von==11), 'instr'] = 3011447
data.loc[(data.von==32) | (data.von==31) | (data.von==33), 'instr'] = 368078
data

,von,nach,aufst,ri,sh,dh,instr
0,10,11,1,0.0000,23.778,-0.019,3011447
1,10,223,1,92.9000,NaN,NaN,3011447
2,10,254,1,346.9069,NaN,NaN,3011447
3,10,33,1,172.2053,23.898,1.104,3011447
4,11,10,1,0.0000,23.779,0.019,3011447
5,11,1247,1,284.4015,NaN,NaN,3011447
6,11,21,1,136.1269,13.040,1.106,3011447
7,11,263,1,103.2949,NaN,NaN,3011447
8,21,11,1,0.0000,13.038,-1.106,3012219
9,21,22,1,262.2336,20.994,-1.100,3012219


Im Anschluss wird die Stochastik der Messdaten über die Instrumententabelle hinzugefügt und überprüft ob (Näherungs-)Koordinaten für alle beteiligten Punkte vorhanden sind:

In [14]:
def check_obs_data(data, instruments, pts):
    """
    Perform Observation Data Consitency Check. 
      1) Checks for instrumen specifications and
         adds standard deviations to observations.
      2) Check for approximate coordinates of 
         involved points.
         
    :param data: observation dataframe
    :param instruments: dataframe of instruments specifications
    :param pts: dataframe of imported points
    
    :returns: extended dataframe of observations with std.dev. 
        and number of observations
    """
    n = 0
    
    for i, r in data.iterrows():
        # first check for instrument specification
        if r.instr not in instruments.seriennr.tolist():
            raise ValueError(r"Instrument specification for '{}' not found".format(r.instr))
        else:
            # and add standard deviations for observations
            if not np.isnan(r.ri):
                data.at[i, 's_ri'] = instruments.at[data.at[i, 'instr'], 's_ri']
                n += 1
                
            if not np.isnan(r.sh):
                data.at[i, 's_sh'] = instruments.at[data.at[i, 'instr'], 's_d']    
                n += 1
        
        # also check for coordinates of involved points
        if r.von not in pts.index.tolist():
            raise ValueError(r"Standpunkt ({}) coordinates not found".format(r.von))
        if r.nach not in pts.index.tolist():
            raise ValueError(r"Zielpunkt ({}) coordinates not found".format(r.nach))
    
    return data, n

In [15]:
data, n = check_obs_data(data, instrumente, punkte)
data

,von,nach,aufst,ri,sh,dh,instr,s_ri,s_sh
0,10,11,1,0.0000,23.778,-0.019,3011447,0.3,2.0
1,10,223,1,92.9000,NaN,NaN,3011447,0.3,NaN
2,10,254,1,346.9069,NaN,NaN,3011447,0.3,NaN
3,10,33,1,172.2053,23.898,1.104,3011447,0.3,2.0
4,11,10,1,0.0000,23.779,0.019,3011447,0.3,2.0
5,11,1247,1,284.4015,NaN,NaN,3011447,0.3,NaN
6,11,21,1,136.1269,13.040,1.106,3011447,0.3,2.0
7,11,263,1,103.2949,NaN,NaN,3011447,0.3,NaN
8,21,11,1,0.0000,13.038,-1.106,3012219,0.3,2.0
9,21,22,1,262.2336,20.994,-1.100,3012219,0.3,2.0


Als Nebenprodukt haben wir gleich die Anzahl der Beobachtungen bestimmt:

In [16]:
n

47

# 2 Ausgleichung vorbereiten

Das Ausgleichungsproblem besteht aus zwei Teilen:

* dem **funktionalen Model**
* dem **stochastischen Modell**

Das **funktionale Modell** beschreibt den deterministischen Zusammenhang zwischen Beobachtungen und Unbekannten und hat die Form:

$$ \boldsymbol{l} + \boldsymbol{v} = \varphi \left( \boldsymbol{x} \right) $$

*Das setzt voraus, dass jede Beobachtung als Funktion der Unbekannten ausgedrückt werden kann und wird als Ausgleichung nach Gauß-Markov Modell oder vermittelnden Beobachtungen bezeichnet. Ist das nicht möglich wird man auf die implizite Formulierung $\varphi \left( \boldsymbol{x}, \boldsymbol{l} + \boldsymbol{v} \right) = 0$ zurückgreifen, was zum allgemeinen Fall der Ausgleichungsrechnung oder Ausgleichung im Gauß-Helmert Modell führt.*

Für die Horizontalstrecke gilt der Zusammenhang:

$$ s_{ij}^h = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2}$$

Für die unorientierte Richtung:

$$ R_{ij} = t_{ij} - o_i = \arctan{\frac{(y_j-y_i)}{x_j-x_j}} - o_i$$

Man sieht schon, dass zusätzlich zu den Koordinaten der Neupunkte pro Standpunkt und Aufstellung mit Richtungsbeobachtung eine Unbekannte hinzukommt.

Das **stochastische Modell** beschreibt die stochastischen Eigenschaften der Eingangsgrößen, sprich der Beobachtungen. Aus den Varianzen (und Kovarianzen) der Beobachtungen ($\boldsymbol{\Sigma_{ll}}$) ergeben sich umgehkehrt proportional die Gewichte (siehe weiter unten $\boldsymbol{P}$-Matrix).


## 2.1 Parameter aufstellen
Die Anzahl der Parameter (Unbekannte) setzt sich einerseits aus den 

* Koordinaten der Neupunkte und den 
* Orientierungsunbekannten der Standpunkte zusammen

Mit der folgenden Funktion bestimmen wir die Parameter und deren Position. Für die Koordinaten gibt es zwei Indizes, jeweils für `y` und `x`. Die Orientierungsunbekannte `o` wird unter der negativen Punktnummer verspeichert.

In [17]:
def get_parameters(punkte, data):
    """
    Define and organize unknown parameters.
    - Coordinates of new points
    - Orientation Unknowns of stations
    
    :param punkte: dataframe of coordinates
    :param data: observation dataframe
    
    :returns: tuple(parameters, number of parameters)
        parameters is a dictionary of parameters and 
        their position within x
    """
    # get coordinate parameters and store their index
    j = 0
    idx = {}
    for i in punkte[punkte['art']=='N'].sort_index().index:
        idx[i] = [j, j+1]
        j += 2
        
    # determine orientation unknowns
    # Assumption: every point has a direction observation and only one station per point
    stdpkt = data.von.unique().tolist()
    for i in stdpkt:
        idx[-i] = [j]
        j += 1
    
    return idx, j

In [18]:
params, u = get_parameters(punkte, data)
params

{10: [0, 1],
 11: [2, 3],
 22: [4, 5],
 31: [6, 7],
 32: [8, 9],
 33: [10, 11],
 -10: [12],
 -11: [13],
 -21: [14],
 -22: [15],
 -31: [16],
 -32: [17],
 -33: [18]}

Die Anzahl der Unbekannten beträgt damit:

In [19]:
u

19

## 2.2 Design Matrix und gekürzter Beobachtungsvektor

Auf Grund der Nichtlinearität des funktionalen Modells müssen wir diese linearisieren, d.h. wir brauchen 

a) Näherungswerte der Unbekannte $\boldsymbol{x}_0$ = Linearisierungstelle, und

b) die Ableitungen der Beobachtungsmodelle nach den Unbekannten, die in der Designmatrix $\boldsymbol{A}$ zusammengefasst werden.

Aus den Näherungswerten kann man zugehörige Beobachtungen berechnen, wie sie beobachtet werden würden.

$$ \boldsymbol{l}_0 = \varphi \left( \boldsymbol{x}_0 \right) $$

Die Differenz zum tatsächlich beobachteten Beobachtungsvektor ist der **gekürzte Beobachtungsvektor**:

$$ \boldsymbol{l} = \boldsymbol{l}_{obs} - \boldsymbol{l}_0 $$

Die Dimension der Designmatrix ($\boldsymbol{A}$), des gekürzten Beobachtungsvektor $\boldsymbol{l}$ und die Kofaktoren der Beobachtungen $\boldsymbol{Q}_{ll}$ kennen wir bereits (Anzahl Unbekannter und Beobachtungen) und können die entsprechenden Matrizzen initialisieren:

In [20]:
A = np.zeros((n, u))
l = np.zeros((n, 1))
Qll = np.zeros((n, n))

In [21]:
A.shape, l.shape, Qll.shape

((47, 19), (47, 1), (47, 47))

### 2.2.1 Hilfsfunktionen

Zuerst brauchen wir noch ein paar **Hilfsfunktionen**. 

Die erste verwenden wir um die Koordinaten eines oder mehrerer Punkte mittels der Punktnummer zu erhalten. 

In [22]:
def get_coordinates(punkte, *pt_names):
    """
    Method to access coordinates in point dataframe.
    
    :param punkte: points dataframe
    :param *pt_names: arbitrary number of point names to 
        query coordinates
    
    :returns: tuple of y,x sorted coordinates 
    """
    coords = ()
    for pt in pt_names:
        coords += punkte.loc[pt].y, punkte.loc[pt].x
    
    return coords

Als Beispiel fragen wir die Koordinaten der Punkte '10' und '11' ab:

In [23]:
get_coordinates(punkte, 10, 11)

(2764.45, 5339745.891, 2784.9109999999996, 5339758.008)

Die zweite soll uns die Indizes (Spalten in der Designmatrix $\boldsymbol{A}$) der Parameter liefern:

In [24]:
def get_params_idx(params, *args):
    """
    Query for index of parameters. If a parameter is not found, Nones
    are added to the list.
    
    :param params: dictionary of parameters (created by :func:get_parameters)
    :param args: arbitrary number of parameter ids
    
    :returns: list of indices within parameter vector
    """
    idx = []
    for p in args:
        if p in params:
            idx += params[p]
        else:
            idx += [None, None]
        
    return idx

Zum Beispiel erhalten wir mit folgendem Aufruf die Positionen der Koordinaten für Punkte '10' und '11', sowie der Orientierungsunbekannten für Standpunkt '10':

In [25]:
get_params_idx(params, 10, 11, -10)

[0, 1, 2, 3, 12]

Für Punkte, die keine Neupunkte sind, wird `None` zurückgegeben.

In [26]:
get_params_idx(params, 10, 223, -10)

[0, 1, None, None, 12]

### 2.2.2 Funktionales Modell

Für die Horizontalstrecke gilt der Zusammenhang:

$$ s_{ij}^h = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2}$$

Für die unorientierte Richtung:

$$ R_{ij} = t_{ij} - o_i = \arctan{\frac{(y_j-y_i)}{x_j-x_i}} - o_i$$

Für die Designmatrix $\boldsymbol{A}$ werden die partiellen Ableitung nach den Unbekannten benötigt, also 4 Terme für die Horizontalstrecke und 5 Terme für die Richtung.

In [27]:
def model_distance(y1, x1, y2, x2, l):
    """
    Functional Model of Distance observation
    """
    l_comp = np.sqrt((y1 - y2) ** 2 + (x1 - x2) ** 2)
    a1 = (y1 - y2) / l_comp
    a2 = (x1 - x2) / l_comp
    a3 = -a1
    a4 = -a2

    l_short = l - l_comp

    return a1, a2, a3, a4, l_short

In [28]:
def model_direction(y1, x1, y2, x2, o1, l):
    """
    Functional Model of Direction observation
    """
    l_comp = np.arctan2((y2 - y1), (x2 - x1)) - o1
    s2 = ((x1 - x2) ** 2 + (y1 - y2) ** 2)
    
    a1 = (x1 - x2) / s2
    a2 = (-y1 + y2) / s2
    a3 = -a1
    a4 = -a2
    a5 = -1

    l_short = l - l_comp
    
    if l_short > 2 * np.pi:
        l_short -= 2 * np.pi
    if l_short < 0:
        l_short += 2 * np.pi

    if abs(l_short) > abs(l_short - 2 * np.pi):
        l_short -= 2 * np.pi

    return a1, a2, a3, a4, a5, l_short

### 2.2.3 Die Matrizzen

Die Designmatrix $\boldsymbol{A}$ wird nun entsprechend den Beobachtungen befüllt, dazu müssen wir über die Zeilen der Beobachtungen iterieren und für jede vorhandene Beobachtung das entsprechend funktionale Modell an der Stelle der Näherungswerte auswerten. D.h. die Koordinaten von Stand- und Zielpunkt werden benötigt.

Beim Befüllen der $\boldsymbol{A}$-Matrix ist es wichtig die entsprechenden Spalten für die jeweiligen Unbekannten zu besetzen. Dazu kann die vorhanden Funktion `get_params_idx()` verwendet werden. Da diese für Nicht-Neupunkte (also Festpunkte) `None` zurückliefert ist dabei schon sicher gestellt, dass die Festpunkte (wie beim **gezwängten Ausgleich** notwendig) nicht an der Ausgleichung teilnehmen, d.h. die jeweiligen Ableitungs-Terme nicht eingefüllt werden.

*Ein anderer Ansatz wäre zuerst eine volle A-Matrix für alle Punkte (und alle Ableitungen) aufzustellen und dann die entsprechenden Spalten der Festpunkte zu entfernen.*

In [29]:
# os = {10: 65.9726, 11: -134.0277, 21: -197.9000, 22: -73.8111, 31: -18.5401, 32: -18.3581, 33: 38.1790}
os = {10: 0., 11: 0., 21: 0., 22: 0., 31: 0., 32: 0., 33: 0.}

In [30]:
def netz2d(data, punkte, params):
    """
    Adjustment Preparation: created Designmatrix :math:`\boldsymbol{A}`, 
    shortened observation vector :math:`\boldsymbol{l}` and 
    Cofactormatrix :math:`\boldsymbol{Q}_{ll}`.
    
    :param data: observation dataframe
    :param punkte: coordinates dataframe
    :param params: dictionary of parameters (created by :func:get_parameters)
    
    :returns: (:math:`\boldsymbol{A}`, :math:`\boldsymbol{l}`, :math:`\boldsymbol{Q}_{ll}`)
    """
    
    def fill_A_row(idx, a0, j):
        """
        Fill elements of matrix **A** at columns **idx** and row **j** with **a0**
        """
        # remove terms for Festpunkte
        a = [x for i, x in enumerate(a0) if idx[i] is not None]
        idx = [x for x in idx if x is not None]            
        A[j,idx] = a
    
    j = 0
    for i, r in data.iterrows():
        # get the coordinates of the two involved points
        pts = get_coordinates(punkte, data.at[i, 'von'] ,data.at[i, 'nach'])
        if not np.isnan(r.ri):
            # there is a direction observation in this row           
            o = os[data.at[i, 'von']]/200*np.pi
            # compute l_short and derivatives
            a0, a1, a2, a3, a4, li = model_direction(*(pts + (o, r.ri/200*np.pi)))
            l[j] = li
            # get columns of parameters
            idx = get_params_idx(params, data.at[i, 'von'], data.at[i,'nach'], -data.at[i, 'von'])            
            fill_A_row(idx, [a0, a1, a2, a3, a4], j)            
            # fill Kofactor-Matrix (at this point it is rather a Variance Matrix)
            Qll[j,j] = (data.at[i, 's_ri']/1e3/200*np.pi)**2
            j += 1
        if not np.isnan(r.sh):
            # ther is a disance observations in this row
            # compute l_short and derivatives
            a0, a1, a2, a3, li = model_distance(*(pts + (r.sh,)))
            l[j] = li
            # get columns of parameters
            idx = get_params_idx(params, data.at[i, 'von'], data.at[i,'nach'])
            fill_A_row(idx, [a0, a1, a2, a3], j) 
            # fill Kofactor-Matrix (at this point it is rather a Variance Matrix)
            Qll[j,j] = (data.at[i, 's_sh']/1e3)**2
            j += 1
    
    return A, l, Qll
            

Im Zuge dieser zentralen Funktion wurde gleich auch die Diagonale der Kofaktormatrix mit der jeweiligen Varianz (abhängig vom Beobachtungstyp) befüllt. Wir gehen hier davon aus, dass alle Beobachtungen von einander unabhängig sind (Nebendiagonalelemente = 0). Das ist eine Vereinfachung und in der Realität nicht der Fall, da z.B. Zentrierfehler Korrelationen zwischen den Beobachtungen eines Standpunktes erzeugen.

In [31]:
A, l, Qll = netz2d(data, punkte, params)
show_matrix(A)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-0.021428,0.0361838,0.021428,-0.0361838,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0
1,-0.86044,-0.509552,0.86044,0.509552,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0010509,0.000792334,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0
3,-0.000710927,0.000145816,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0
4,0.0345443,-0.0236161,0,0,0,0,0,0,0,0,-0.0345443,0.0236161,-1,0,0,0,0,0,0
5,0.564367,0.825524,0,0,0,0,0,0,0,0,-0.564367,-0.825524,0,0,0,0,0,0,0
6,-0.021428,0.0361838,0.021428,-0.0361838,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0
7,-0.86044,-0.509552,0.86044,0.509552,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0.000316759,0.000313084,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0
9,0,0,-0.0766519,0.00251154,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0


Eigentlich ist zu diesem Zeitpunkt `Qll` noch die (Ko)Varianz-Matrix ($\boldsymbol{\Sigma}_{ll}$). Zur Kofaktor-Matrix wird sie erst, wenn wir die Varianz der Gewichtseinheit apriori herausheben:

$$ \sigma_0^2 \cdot \boldsymbol{Q}_{ll} = \boldsymbol{\boldsymbol{\Sigma}_{ll}} $$

In [32]:
s02_prio = 1.
Qll = Qll / s02_prio
s02_prio

1.0

# 3 Die Ausgleichung

## 3.1 Die Lösung

Die Gewichtsmatrix $\boldsymbol{P}$ ergibt sich aus der Inversion der Kofaktormatrix der Beobachtungen

$$\boldsymbol{P} = {\boldsymbol{Q}_{ll}}^{-1}$$

In [33]:
P = np.linalg.inv(Qll)
show_matrix(P[:4,:4])

,0,1,2,3
0,4.50316e+10,0,0,0
1,0,250000,0,0
2,0,0,4.50316e+10,0
3,0,0,0,4.50316e+10


Die Lösung des Gleichungssystem erfolgt über die Normalgleichungsmatrix $\boldsymbol{N} = \boldsymbol{A}^T \; \boldsymbol{P} \; \boldsymbol{A}$ und

$$\boldsymbol{x} = \boldsymbol{N}^{-1}\;\boldsymbol{A}^T\;\boldsymbol{P}\;\boldsymbol{l}$$

In [34]:
N = A.T @ P @ A
show_matrix(N)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,4.01622e+08,-1.17587e+08,-4.17237e+07,6.96111e+07,0,0,-2.52193e+08,-2.52324e+07,0,0,-1.07632e+08,7.32408e+07,-6.05955e+08,9.64939e+08,0,0,3.36995e+09,0,-1.55559e+09
1,-1.17587e+08,1.71424e+08,6.96111e+07,-1.18047e+08,0,0,-2.52324e+07,-2.77706e+06,0,0,7.32408e+07,-5.05709e+07,-6.0819e+08,-1.62942e+09,0,0,3.37504e+08,0,1.06347e+09
2,-4.17237e+07,6.96111e+07,5.70907e+08,-8.69231e+07,0,0,0,0,0,0,0,0,-9.64939e+08,2.49449e+09,3.45176e+09,0,0,0,0
3,6.96111e+07,-1.18047e+08,-8.69231e+07,1.19122e+08,0,0,0,0,0,0,0,0,1.62942e+09,1.51371e+09,-1.13099e+08,0,0,0,0
4,0,0,0,0,6.35616e+07,-1.00778e+08,-5.46184e+06,9.03298e+06,0,0,0,0,0,0,-1.13991e+09,-1.09065e+09,-4.87349e+08,0,0
5,0,0,0,0,-1.00778e+08,1.62671e+08,9.03298e+06,-1.59056e+07,0,0,0,0,0,0,1.81677e+09,1.84758e+09,8.44657e+08,0,0
6,-2.52193e+08,-2.52324e+07,0,0,-5.46184e+06,9.03298e+06,3.19365e+08,-9.40131e+07,-2.96379e+07,5.13749e+07,0,0,0,0,8.44609e+08,0,-2.84503e+09,-8.11667e+08,0
7,-2.52324e+07,-2.77706e+06,0,0,9.03298e+06,-1.59056e+07,-9.40131e+07,2.19919e+08,5.13749e+07,-9.10824e+07,0,0,0,0,-1.57407e+09,0,-1.32374e+09,1.4311e+09,0
8,0,0,0,0,0,0,-2.96379e+07,5.13749e+07,1.58763e+08,2.56056e+07,-1.29096e+08,-7.69794e+07,0,0,0,0,8.11667e+08,-8.59509e+08,-1.70404e+09
9,0,0,0,0,0,0,5.13749e+07,-9.10824e+07,2.56056e+07,1.37692e+08,-7.69794e+07,-4.65808e+07,0,0,0,0,-1.4311e+09,-2.41676e+09,-1.02006e+09


In [35]:
x = np.linalg.solve(N, A.T @ P @ l)

Die Verbesserung folgen aus

$$\boldsymbol{v} = \boldsymbol{A}\;\boldsymbol{x} - \boldsymbol{l}$$

In [36]:
v = A @ x - l

## 3.2 Ausgeglichene Parameter

Für die asugeglichenen Parameter sind die Zuschläge zu addieren:

In [37]:
def update_parameters(params, punkte, x):
    """
    Update coordinates dataframe with adjusted coordinates and create 
    Dataframe for orientation unknowns.
    
    :param params: dictionary of parameters (created by :func:get_parameters)
    :param punkte: dataframe of coordinates
    :param x: ajdusted parameters vector
    
    :returns: updated coordinates dataframe and orientation unknowns dataframe
    """
    ori = punkte[['art']].copy()
    for p, idx in params.items():
        if p > 0:
            punkte.at[p, 'dy'] = x[idx[0]]
            punkte.at[p, 'dx'] = x[idx[1]]
            punkte.at[p, 'y_hat'] = punkte.at[p, 'y'] + x[idx[0]]
            punkte.at[p, 'x_hat'] = punkte.at[p, 'x'] + x[idx[1]]
        else:
            ori.at[-p, 'ori'] = x[idx[0]]/np.pi*200
    
    return punkte, ori[['ori']][~np.isnan(ori.ori)]

punkte, ori = update_parameters(params, punkte, x)

Das ergibt die ausgeglichenen Orientierungsunbekannten und Neupunktskoordinaten:

In [38]:
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(punkte[punkte.art=='N'])
    display(ori)

,y,x,h,art,dy,dx,y_hat,x_hat
pt,,,,,,,,
10,2764.4500,5339745.8910,nan,N,-0.0008,0.0054,2764.4492,5339745.8964
11,2784.9110,5339758.0080,nan,N,-0.0029,-0.0002,2784.9081,5339758.0078
22,2803.1230,5339782.1990,nan,N,0.0037,-0.0003,2803.1267,5339782.1987
31,2763.1250,5339759.1210,nan,N,-0.0004,0.0006,2763.1246,5339759.1216
32,2739.3170,5339745.6180,nan,N,0.0001,0.0039,2739.3171,5339745.6219
33,2750.9630,5339726.1630,nan,N,-0.0018,0.0056,2750.9612,5339726.1686


,ori
pt,
10,65.9725
11,-134.0277
21,-197.9002
22,-135.6647
31,-18.5390
32,-18.3580
33,38.1788


## 3.3 Ausgeglichene Beobachtungen

Für die ausgeglichenen Beobachtungen sind ebenfalls nur die Verbesserungen anzubringen.

In [39]:
def update_obs(data, v):
    """
    Add adjusted values to observation dataframe (v and l_hat)
    
    :param data: observations dataframe
    :param v: residuals vector of adjustment
    
    :returns: updated observations dataframe
    """
    j = 0
    for i, r in data.iterrows():
        if not np.isnan(r.ri):
            data.at[i, 'ri_v'] = v[j]/np.pi*200*1e3
            data.at[i, 'ri_hat'] = data.at[i, 'ri'] + v[j]/np.pi*200
            j += 1
        if not np.isnan(r.sh):
            data.at[i, 'sh_v'] = v[j]*1e3
            data.at[i, 'sh_hat'] = data.at[i, 'sh'] + v[j]
            j += 1
        
    return data
    
data = update_obs(data, v)
display(data)

,von,nach,aufst,ri,sh,dh,instr,s_ri,s_sh,ri_v,ri_hat,sh_v,sh_hat
0,10,11,1,0.0000,23.778,-0.019,3011447,0.3,2.0,-0.179978,-0.000180,-2.989273,23.775011
1,10,223,1,92.9000,NaN,NaN,3011447,0.3,NaN,0.135685,92.900136,NaN,NaN
2,10,254,1,346.9069,NaN,NaN,3011447,0.3,NaN,-0.426387,346.906474,NaN,NaN
3,10,33,1,172.2053,23.898,1.104,3011447,0.3,2.0,0.470679,172.205771,-0.066451,23.897934
4,11,10,1,0.0000,23.779,0.019,3011447,0.3,2.0,0.100117,0.000100,-3.989273,23.775011
5,11,1247,1,284.4015,NaN,NaN,3011447,0.3,NaN,-2.337631,284.399162,NaN,NaN
6,11,21,1,136.1269,13.040,1.106,3011447,0.3,2.0,0.267507,136.127168,-0.693195,13.039307
7,11,263,1,103.2949,NaN,NaN,3011447,0.3,NaN,1.970007,103.296870,NaN,NaN
8,21,11,1,0.0000,13.038,-1.106,3012219,0.3,2.0,-0.339787,-0.000340,1.306805,13.039307
9,21,22,1,262.2336,20.994,-1.100,3012219,0.3,2.0,0.345048,262.233945,4.902553,20.998903


## 3.4 Globaltest & Kovarianzen Aposteriori

Für die **Varianz der Gewichtseinheit aposteriori** wird die gewichtete Verbesserungsquadratsumme herangezogen:

$$ s_0^2 = \frac{\boldsymbol{v}^T \, \boldsymbol{P} \, \boldsymbol{v}}{f} $$

wobei sich der Freiheitsgrad $f$ aus der Überbestimmung ergibt: $f = n - u$

Für den **Globaltest** der Ausgleichung überprüft man das Verhältnis aus $\dfrac{\hat{s}_0^2}{\sigma_0^2}$. Diese Testgröße folgt einer $\chi^2$-Verteilung.

$$ T_G = f\,\dfrac{\hat{s}_0^2}{\sigma_0^2} \leq \chi^2_{0.95,f} $$ 

In [40]:
from scipy.stats import chi2

f = n - u
alpha = 0.05
s02_post = (v.T @ P @ v)[0,0]/f

T = s02_post/s02_prio
F = chi2.ppf(1-alpha, f)/f

print('{:.1f} ≤ {:.1f}'.format(T, F))
print('Globaltest erfolgreich: ', T <= F)

23.7 ≤ 1.5
Globaltest erfolgreich:  False


Die Kofaktoren aposteriori sind:

$$ 
\begin{align}
\boldsymbol{Q}_{\hat{x}\hat{x}} & = \boldsymbol{N}^{-1} \\
\boldsymbol{Q}_{\hat{l}\hat{l}} & = \boldsymbol{A} \, \boldsymbol{N}^{-1} \, \boldsymbol{A}^T \\
\boldsymbol{Q}_{vv} & = \boldsymbol{Q}_{ll} - \boldsymbol{Q}_{\hat{l}\hat{l}}
\end{align}
$$

In [41]:
Qxx = np.linalg.inv(N)
Qllp = A @ np.linalg.inv(N) @ A.T
Qvv = Qll - Qllp
Sll = s02_post * Qllp
Sxx = s02_post * Qxx
Svv = s02_post * Qvv

Dann können wir die Varianzinformationenen der Parameter ergänzen:

In [42]:
def add_parameter_var(params, punkte, ori, Sxx):
    """
    Add a-posteriori stochastic information to parameters
    
    :param params: dictionary of parameters (created by :func:get_parameters)
    :param punkte: dataframe of coordinates
    :param ori: dataframe of orientation unknowns
    :param Sxx: Covariance Matrix from adjustment
    
    :returns: updated dataframe of coordinates and orientation unknowns
    """
    s = np.sqrt(np.diag(Sxx))
    for p, idx in params.items():
        if p > 0:
            punkte.at[p, 's_y'] = s[idx[0]]*1e3
            punkte.at[p, 's_x'] = s[idx[1]]*1e3
        else:
            ori.at[-p, 's_o'] = s[idx[0]]/np.pi*200*1e3
            
    return punkte, ori

Für die Ausgabe kann man sich noch ein wenig mit dem Styling für ein pandas DataFrame spielen:

(siehe https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

In [43]:
punkte, ori = add_parameter_var(params, punkte, ori, Sxx)

def color_value(value, threshold=1., absolute=True):
    """
    Colors elements in a dataframe
    according to value
    
    :param threshold: treshold to exceed to color red, 
        orange if bigger than 2/3 of threshold 
    :param absolute: look at absolute values
    """
    if absolute:
        value = abs(value)
        
    if value > threshold:
        color = 'red'
    elif value > threshold/3*2:
        color = 'orange'
    else:
        color = 'black'

    return 'color: %s' % color
    
# define float formats and column names
punkte.rename(columns={"y": "y [m]", "x": "x [m]", "dy": "dy [m]", "dx": "dx [m]",
                      "x_hat": "x_hat [m]", "y_hat": "y_hat [m]", "s_x": "s_x [mm]", "s_y": "s_y [mm]"}, inplace=True)
float_cols = [c for c in punkte.dtypes.index  if 'float' in str(punkte.dtypes[c])]
s_p = punkte[punkte.art=='N'].style.format(dict(zip(float_cols,  [lambda x: "{:.3f}".format(x)]*10)))


ori.rename(columns={"ori": "ori [gon]", "s_o": "s_o [mgon]"}, inplace=True)
float_cols = [c for c in ori.dtypes.index  if 'float' in str(ori.dtypes[c])]
s_o = ori.style.format(dict(zip(float_cols,  [lambda x: "{:.4f}".format(x)]*10)))
    
with pd.option_context('display.float_format', '{:.4f}'.format):
    display(ipy.display.HTML('<font size="4"><strong>Neupunktskoordinaten</strong></font>'))
    display(s_p.applymap(color_value, subset=['dx [m]','dy [m]'], threshold=0.01))
    display(ipy.display.HTML('<font size="4"><strong>Orientierungen</strong></font>'))
    display(s_o.applymap(color_value, subset=['s_o [mgon]'], threshold=0.9))
    

,y [m],x [m],h,art,dy [m],dx [m],y_hat [m],x_hat [m],s_y [mm],s_x [mm]
pt,,,,,,,,,,
10,2764.450,5339745.891,nan,N,-0.001,0.005,2764.449,5339745.896,3.749,4.949
11,2784.911,5339758.008,nan,N,-0.003,-0.000,2784.908,5339758.008,0.290,4.744
22,2803.123,5339782.199,nan,N,0.004,-0.000,2803.127,5339782.199,4.779,3.024
31,2763.125,5339759.121,nan,N,-0.000,0.001,2763.125,5339759.122,3.726,2.061
32,2739.317,5339745.618,nan,N,0.000,0.004,2739.317,5339745.622,5.283,2.970
33,2750.963,5339726.163,nan,N,-0.002,0.006,2750.961,5339726.169,4.770,5.361


,ori [gon],s_o [mgon]
pt,,
10,65.9725,0.8798
11,-134.0277,0.8640
21,-197.9002,0.7065
22,-135.6647,0.9402
31,-18.5390,0.9811
32,-18.3580,0.6740
33,38.1788,1.0826


Und für die Beobachtungen ebenfalls die Standardabweichungen a-posteriori un die normierten Verbesserungen $ nv = \frac{v}{\sigma_{v}} $:

In [44]:
def add_obs_var(data, Sll, Svv):
    """
    Add a-posteriori stochastic information to observations/
    
    :param data: observations dataframe
    :param Sll: Covariance Matrix of observations from adjustment
    :param Svv: Covariance Matrix of residuals from adjustment
    
    :returns: updated observations dataframe
    """
    j = 0
    ll = np.sqrt(np.diag(Sll))
    vv = np.sqrt(np.diag(Svv))
    for i, r in data.iterrows():
        if not np.isnan(r.ri):
            data.at[i, 's_ri_p'] = ll[j]/np.pi*200*1e3
            data.at[i, 'ri_nv'] = np.abs(data.at[i, 'ri_v']) / (vv[j]/np.pi*200*1e3)
            j += 1
        if not np.isnan(r.sh):
            data.at[i, 's_sh_p'] = ll[j]*1e3
            data.at[i, 'sh_nv'] = np.abs(data.at[i, 'sh_v']) / (vv[j]*1e3)
            j += 1
        
    return data

In [45]:
data = add_obs_var(data, Sll, Svv)

In [46]:
def color_row(s, column=None, threshold=1.):  
    """
    Colors a row based on the value of the specified column.
    
    :param s: pandas Series
    :param column: column name to compare with threshold
    :param threshold: define colorization  by
        - values > threshold are colored red
        - values > 2/3 threshold are colored orange
        
    """
    if column is None:
        return ['color: black']*len(s)
    
    if s[column] > threshold:
        return ['color: red']*len(s)
    elif s[column] > 2/3*threshold:
        return ['color: orange']*len(s)
    else:
        return ['color: balck']*len(s)

In [47]:
ris = data.loc[~np.isnan(data.ri), ['von', 'nach', 'aufst', 'ri', 's_ri', 'ri_v', 'ri_hat', 's_ri_p', 'ri_nv']]

show_dataframe(ris,
               title='Richtungen', 
               headers={"ri": "ri [gon]","ri_hat": "ri_hat [gon]", "s_ri": "s_ri [mgon]",
                   "ri_v": "ri_v [mgon]", "s_ri_p": "s_ri_p [mgon]", "ri_nv": "ri_nv []"},
               formats={'ri': lambda x: "{:.4f}".format(x), 'ri_hat': lambda x: "{:.4f}".format(x),
                       'ri_v': lambda x: "{:.2f}".format(x),  's_ri_p': lambda x: "{:.2f}".format(x),
                       'ri_nv': lambda x: "{:.1f}".format(x)},
               styler=color_row, styler_kwargs={'column':'ri_nv', 'threshold':1.96, 'axis':1})

,von,nach,aufst,ri [gon],s_ri [mgon],ri_v [mgon],ri_hat [gon],s_ri_p [mgon],ri_nv []
0,10,11,1,0.0000,0.3,-0.18,-0.000179978,1.17,0.2
1,10,223,1,92.9000,0.3,0.14,92.9001,0.93,0.1
2,10,254,1,346.9069,0.3,-0.43,346.906,0.90,0.4
3,10,33,1,172.2053,0.3,0.47,172.206,1.21,0.6
4,11,10,1,0.0000,0.3,0.10,0.000100117,1.17,0.1
5,11,1247,1,284.4015,0.3,-2.34,284.399,0.87,2.0
6,11,21,1,136.1269,0.3,0.27,136.127,1.15,0.3
7,11,263,1,103.2949,0.3,1.97,103.297,0.86,1.7
8,21,11,1,0.0000,0.3,-0.34,-0.000339787,1.15,0.4
9,21,22,1,262.2336,0.3,0.35,262.234,1.13,0.4


In [48]:
shs = data.loc[~np.isnan(data.sh), ['von', 'nach', 'aufst', 'sh', 's_sh', 'sh_v', 'sh_hat', 's_sh_p', 'sh_nv']]

show_dataframe(shs,
               title='Strecken', 
               headers={"sh": "sh [m]","sh_hat": "sh_hat [m]", "s_sh": "s_sh [mm]",
                   "sh_v": "sh_v [mm]", "s_sh_p": "s_sh_p [mm]", "sh_nv": "sh_nv []"},
               formats={'sh': lambda x: "{:.4f}".format(x), 'sh_hat': lambda x: "{:.4f}".format(x),
                       'sh_v': lambda x: "{:.2f}".format(x),  's_sh_p': lambda x: "{:.2f}".format(x),
                       'sh_nv': lambda x: "{:.1f}".format(x)},
               styler=color_row, styler_kwargs={'column':'sh_nv', 'threshold':1.96, 'axis':1})

,von,nach,aufst,sh [m],s_sh [mm],sh_v [mm],sh_hat [m],s_sh_p [mm],sh_nv []
0,10,11,1,23.7780,2,-2.99,23.775,4.39,0.3
3,10,33,1,23.8980,2,-0.07,23.8979,5.12,0.0
4,11,10,1,23.7790,2,-3.99,23.775,4.39,0.5
6,11,21,1,13.0400,2,-0.69,13.0393,4.75,0.1
8,21,11,1,13.0380,2,1.31,13.0393,4.75,0.2
9,21,22,1,20.9940,2,4.90,20.9989,5.64,0.6
13,21,31,1,25.2080,2,0.75,25.2088,4.23,0.1
14,22,21,1,20.9950,2,3.90,20.9989,5.64,0.5
18,31,21,1,25.2100,2,-1.25,25.2088,4.23,0.1
19,31,32,1,27.3700,2,-1.36,27.3686,4.36,0.2


# 9 Future Notes

* Colored Matrix Visualization: https://stackoverflow.com/a/40890587
* Improve pandas output: https://stackoverflow.com/a/20937592